## 关于Baseline参考文章
>- **🍨 Baseline参考[🔗365天深度学习训练营](https://www.heywhale.com/home/activity/detail/62d0e7de05e9c8e1d50a90fa/content/4) 内部限免文章（版权归 *K同学啊* 所有）**
>- **🍦 参考文章地址： [🔗运动鞋品牌识别](https://www.heywhale.com/mw/project/62feef0af31025b77757087a)**
>- **🍖 参考文章作者：[K同学啊 | 接辅导、程序定制](https://mp.weixin.qq.com/s/k-vYaC8l7uxX51WoypLkTw)**

# 👇本周任务挑战
**5天学习计划-第5周**
> - 选题：👟运动鞋品牌识别
> - 难度：新手入门🌟
> - 语言：Python3、TensorFlow2
> - 时间：8月22-8月26日

**🍺要求：**

> - 跑通程序
> - 理解卷积的运算过程
> - 验证集准确率达到82%

**支持数据**
[运动鞋数据集](https://www.heywhale.com/mw/dataset/62feeea455804632ba14168d/file)

# 代码流程

## 设置GPU

In [2]:
from PIL import Image
import numpy as np

from tensorflow import keras
from tensorflow.keras import layers,models,Model
import os, PIL, pathlib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Dropout, Flatten, Dense, \
    GlobalAveragePooling2D
gpus = tf.config.list_physical_devices("GPU")

if gpus:
    gpu0 = gpus[0]                                        #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True)  #设置GPU显存用量按需使用
    tf.config.set_visible_devices([gpu0],"GPU")
    
gpus

[]

## 从图片属性与卷积核一次运算过程

- 从下面的数据结构可以看出，训练集样本较少，只有578张，模型可能会**存在拟合现象，可以适当的数据增强来增大训练数据集的丰富度**。


 - 图片的大小为240x240,深度为3，意味着输入层的深度也为3，第一层的**卷积核深度为3**,当前层的**卷积核个数**，会决定下一层的深度

**注意**
> 因为上一层的深度为已知，且等于当前卷积核深度，所以在定义卷积核时，不需要传入卷积核的深度，只需要给出**卷积核的长宽和个数**（bias另说)。
![covn_core.drawio.png](https://raw.githubusercontent.com/fisheeped/pcs/hf/covn_core.drawio.png)

- 像素点的大小的色值超过1，可以**规范化数据，降低极值和噪声的方差影响**

In [3]:
data_dir = "./shoes"

data_dir = pathlib.Path(data_dir)

In [4]:
image_count = len(list(data_dir.glob('*/*/*.jpg')))
print("图片总数为：",image_count)

图片总数为： 578


In [5]:
shoes = list(data_dir.glob('test/*/*.jpg'))
img = PIL.Image.open(str(shoes[0]))
img_height, img_width, img_deep=np.asarray(img).shape
np.asarray(img).shape

(240, 240, 3)

In [6]:
np.array(img)

array([[[231, 231, 231],
        [231, 231, 231],
        [231, 231, 231],
        ...,
        [231, 231, 231],
        [231, 231, 231],
        [231, 231, 231]],

       [[231, 231, 231],
        [231, 231, 231],
        [231, 231, 231],
        ...,
        [231, 231, 231],
        [231, 231, 231],
        [231, 231, 231]],

       [[231, 231, 231],
        [231, 231, 231],
        [231, 231, 231],
        ...,
        [231, 231, 231],
        [231, 231, 231],
        [231, 231, 231]],

       ...,

       [[231, 231, 231],
        [231, 231, 231],
        [231, 231, 231],
        ...,
        [231, 231, 231],
        [231, 231, 231],
        [231, 231, 231]],

       [[231, 231, 231],
        [231, 231, 231],
        [231, 231, 231],
        ...,
        [231, 231, 231],
        [231, 231, 231],
        [231, 231, 231]],

       [[231, 231, 231],
        [231, 231, 231],
        [231, 231, 231],
        ...,
        [231, 231, 231],
        [231, 231, 231],
        [231, 231, 231]]

## 拆分数据（基于tensorflow底层逻辑的分享)
定义batch最好是32的倍数，以为显卡的一个warp的core数是32（参考**cuda并行运算逻辑**），故最小并行数为32。

### shuffle
由于**shuffle是在buffer**中进行的，并且shuffle的高度和seed的大小有关，所以seed值太大，会占用更多的内存。
> 因考虑随机性，故seed大小最好不要人工干预。   
> 其次shuffle算法本省就属于伪随机，越靠前的数据越容易取出。

### prefetch
关于prefetch**数据预取**，tensorflow源码其实是在初始化之后，GPU并行运算时，CPU在异步预取数据，通过pcie总线，提前将数据下一轮数据准备在GPU中。简单理解就是CPU和GPU两个线程同时运行。相似的操作还有**tf.function**装饰器。  
> 下面的PPT是为**泰山参加Nvidia**并行运算培训关于prefetch部分的内容。  

In [13]:
%%HTML

<div align="center"><iframe src="https://view.officeapps.live.com/op/view.aspx?src=https://developer.download.nvidia.com/training/courses/C-AC-01-V1/AC_UM_NVPROF-zh/NVPROF_UM_2-zh.pptx" frameborder="0" width="900" height="550" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe></div>

### cuda和tensorflow关系
无论是tensorflow还是torch,panddle,cafe等等，都是基于cuda的逻辑上建的房子，建议有空可以了解一下。

In [7]:
batch_size =32

In [8]:
train_val = list(data_dir.iterdir())
train_val

[PosixPath('shoes/train'), PosixPath('shoes/test')]

In [9]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_val[0],
    seed=31,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 502 files belonging to 2 classes.


In [10]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_val[1],
    seed=31,
    image_size=(img_height, img_width),
    batch_size=batch_size)


Found 76 files belonging to 2 classes.
